- 回帰分析
    - 重回帰モデル
    - Ridge回帰モデル
    - Lasso回帰モデル
    - ElasticNet

- 分析流れ
    - 準備
    - モデル構築・学習
    - 評価

In [1]:
# read dataset
from sklearn.datasets import load_boston
boston = load_boston() 

In [2]:
import pandas as pd 

# to dataframe
boston_df = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_df['MEDV'] = boston.target
boston_df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


***データセットの意味***

・CRIM : 犯罪発生率  
・ZN : 25,000平方フィート以上の住宅区画の割合  
・INDUS : 非小売業種の土地面積の割合  
・CHAS : チャールズ川沿いかを表す0か1かの変数  
・NOX : 窒素酸化物の濃度  
・RM : 平均部屋数  
・AGE : 1940年より前に建てられた建物の割合  
・DIS : 5つのボストンの雇用施設への重み付き距離  
・RAD :  高速道路へのアクセスのしやすさ  
・TAX : 10,000ドルあたりの不動産税率  
・PTRATIO : 生徒と教師の割合  
・B : 黒人の割合  
・LSTAT : 低所得者の割合  
・MEDV : 住宅価格の中央値  

**<span style="color: red; ">CRIMからLSTATまでの変数(説明変数)を用いてMEDV(住宅価格の中央値)を予測するモデルを構築します</span>**

### 重回帰モデル

***準備***

In [3]:
# split train data and test data
from sklearn.model_selection import train_test_split #データセットを分割してくれる関数

# X
X = boston_df.drop("MEDV", axis = 1)

# standard
from sklearn import preprocessing
sc=preprocessing.StandardScaler()
sc.fit(X)
X=sc.transform(X)

# Y
Y = boston_df["MEDV"]

#split train data and test data to 8:2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, test_size = 0.2, random_state = 0)

***学習***

In [4]:
# read model
from sklearn.linear_model import LinearRegression 

lr = LinearRegression() 

In [5]:
#learning
lr.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

学習結果

In [6]:
# coefficient
lr.coef_

array([-1.02638248,  1.0433458 ,  0.03759363,  0.59396238, -1.86651867,
        2.60322635, -0.08776804, -2.91646482,  2.12402208, -1.85033055,
       -2.26212378,  0.73967912, -3.5155841 ])

In [7]:
#intercept
lr.intercept_

22.480352884751216

つまり目的変数(住宅価格)は以下の式で学習(近似)されたことが分かります  
**MEDV = -1.0203819×<span style="color: blue; ">CRIM</span>  
  +1.05193879×<span style="color: blue; ">ZN</span>  
  +0.431598266×<span style="color: blue; ">INDUS</span>  
  +1.26419037×<span style="color: blue; ">CHAS</span>  
  -2.16465050×<span style="color: blue; ">NOX</span>  
  +2.61426417 ×<span style="color: blue; ">RM</span>  
  -0.00347475×<span style="color: blue; ">AGE</span>  
  -3.44219665×<span style="color: blue; ">DIS</span>  
  +2.38778178×<span style="color: blue; ">RAD</span>  
  -1.88719563×<span style="color: blue; ">TAX</span>  
  -2.65803807×<span style="color: blue; ">PTRATIO</span>  
  +0.612847149×<span style="color: blue; ">B</span>  
-4.06696488×<span style="color: blue; ">LSTAT</span>   
+<span style="color: green; ">22.565486698749393<span>(切片)**

標準偏回帰係数の値の大きさによって目的変数への影響度がわかります。(絶対値が大きいほど影響度が強い、t-値を見る場合もあります)  
今回の場合はLSTAT, DIS, PTRATIOが影響度が高いと言えますが、重回帰では***多重共線性***に気を付けなければならないので慎重に  
結果を考察しなければなりません

***評価***  
評価データを用いて学習結果の当てはまりがどの程度のものなのか評価します

In [8]:
#predict
Y_pred = lr.predict(X_test)

評価データの目的変数と予測した値を比べていきます

1.平方二乗誤差

In [9]:
# read the model of mean_squared_error
from sklearn.metrics import mean_squared_error 

print('MSE train data: ', mean_squared_error(Y_train, lr.predict(X_train))) 
print('MSE test data: ', mean_squared_error(Y_test, Y_pred)) 

MSE train data:  19.326470203585725
MSE test data:  33.448979997676545


2.決定係数

In [10]:
# read the model of R^2
from sklearn.metrics import r2_score 

print('r^2 train data: ', r2_score(Y_train, lr.predict(X_train)))
print('r^2 test data: ', r2_score(Y_test, Y_pred))

r^2 train data:  0.7730135569264234
r^2 test data:  0.5892223849182506


MSEは値が小さいほど、決定係数は値が1に近いほど精度が良いモデルであると言えます  
学習データと評価データとの値を比べると、学習データの方が精度がかなり良くなってしまっています。⇒<span style="color: red; ">汎化性能が低い</span>  
対策案:標準偏回帰係数の符号、相関係数等を見て説明変数の数を減らす

### Ridge回帰モデル

***学習***

In [11]:
# read model of ridge
from sklearn.linear_model import Ridge #sklearnのRidge回帰用のモデル

In [12]:
# read model
ridge = Ridge(alpha = 1, random_state=0) 

alphaは正規化項の値で大きくすると簡易的なモデルになります。(0の場合、重回帰と同じ条件)  
交差検証などをしてチューニングするべきハイパーパラメータです。

In [13]:
# learning
ridge.fit(X_train, Y_train)

Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=0, solver='auto', tol=0.001)

In [14]:
# cofficient
ridge.coef_

array([-1.01649357,  1.02733437,  0.01192747,  0.59818575, -1.83082046,
        2.61307788, -0.09465198, -2.88262602,  2.04774814, -1.78279739,
       -2.25280667,  0.73945729, -3.50110119])

In [15]:
#intercept
ridge.intercept_

22.481511182464597

***評価***

In [16]:
#predict
Y_pred = ridge.predict(X_test)

1.平方二乗誤差

In [17]:
# print the MSE
print('MSE train data: ', mean_squared_error(Y_train, ridge.predict(X_train))) 
print('MSE test data: ', mean_squared_error(Y_test, Y_pred)) 

MSE train data:  19.32773068803948
MSE test data:  33.53679721129213


2.決定係数

In [18]:
# print the R^2
print('r^2 train data: ', r2_score(Y_train, ridge.predict(X_train)))
print('r^2 test data: ', r2_score(Y_test, Y_pred))

r^2 train data:  0.7729987527288801
r^2 test data:  0.5881439261558417


**<span style="color: blue; ">重回帰と比べてわずかに汎化性が改善できず<span>**

### Lasso回帰モデル

***学習***

In [19]:
# read the model of Lasso regression
from sklearn.linear_model import Lasso 

In [20]:
# read the model
lasso = Lasso(alpha = 0.5, random_state=0) 

alphaは正規化項の値で大きくすると簡易的なモデルになります。(0の場合、重回帰と同じ条件)  
交差検証などをしてチューニングするべきハイパーパラメータです。

In [21]:
#learning
lasso.fit(X_train, Y_train)

Lasso(alpha=0.5, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=0,
   selection='cyclic', tol=0.0001, warm_start=False)

In [22]:
# coefficient
lasso.coef_

array([-0.2022351 ,  0.        , -0.        ,  0.34586306, -0.        ,
        2.85095977, -0.        , -0.        , -0.        , -0.24202672,
       -1.99201487,  0.41276179, -3.49325645])

Lassoの特徴として係数が0の変数が現れます

In [23]:
#intercept
lasso.intercept_

22.551646497220098

***評価***

In [24]:
#learning
Y_pred = lasso.predict(X_test)

1.平方二乗誤差

In [25]:
# print MSE
print('MSE train data: ', mean_squared_error(Y_train, lasso.predict(X_train))) 
print('MSE test data: ', mean_squared_error(Y_test, Y_pred)) 

MSE train data:  22.891090238872952
MSE test data:  39.801317909560716


2.決定係数

In [26]:
#print R^2
print('r^2 train data: ', r2_score(Y_train, lasso.predict(X_train)))
print('r^2 test data: ', r2_score(Y_test, Y_pred))

r^2 train data:  0.731147638618768
r^2 test data:  0.5112110907676243


***<span style="color: blue; ">重回帰・Ridgeと比べると精度は下がっていますが、だいぶ汎化性能を獲得できています<span>***

### ElasticNet

In [27]:
# read the model of ElasticNet
from sklearn.linear_model import ElasticNet 

In [28]:
# read model
en = ElasticNet(alpha=0.5, l1_ratio=0.5, random_state=0) 

alphaは正規化項の値で大きさで、l1_ratioはL1正規化項の割合の大きさです。  
交差検証などをしてチューニングするべきハイパーパラメータです

In [29]:
#learning
en.fit(X_train, Y_train)

ElasticNet(alpha=0.5, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=0, selection='cyclic', tol=0.0001, warm_start=False)

In [30]:
#cofficient
en.coef_

array([-0.49925771,  0.31539313, -0.32684879,  0.5143696 , -0.40817152,
        2.6109783 , -0.        , -0.58160518, -0.        , -0.42786213,
       -1.76440804,  0.55450758, -2.7311829 ])

In [31]:
# intercept
en.intercept_

22.524812764571706

***評価***

In [32]:
#predict
Y_pred = en.predict(X_test)

1.平方二乗誤差

In [33]:
# MSE
print('MSE train data: ', mean_squared_error(Y_train, en.predict(X_train))) #学習データのMSE
print('MSE test data: ', mean_squared_error(Y_test, Y_pred)) #評価データのMSE

MSE train data:  22.640342097660334
MSE test data:  40.04401182359337


2.決定係数

In [34]:
#R^2
print('r^2 train data: ', r2_score(Y_train, en.predict(X_train)))
print('r^2 test data: ', r2_score(Y_test, Y_pred))

r^2 train data:  0.7340926372699239
r^2 test data:  0.508230634346887


***<span style="color: blue; ">Lassoよりも評価データに対する精度できず<span>***